# Python function to create ome.tif files form multiple [z-stack]directories 


In [3]:
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import tifffile
from tifffile import imread
from matplotlib.backends.backend_pdf import PdfPages
import glob, os, re

from aicsimageio import AICSImage, imread, writers
from skimage import color, io

In [4]:
def Get_ome_tif_file(files = glob.glob('Fch+Bch/24hr_0uM_V_630X1/*.tif') , SaveIMG = "tr_1/One-Ome_tif/24hr_0uM_V_630X1.ome.tif"):

    
    files.sort()
    Fluro_images = []
    Grayscale_Fluro_images = []
    Brightfield_images = []

    for i in range(0, len(files),3):
        Fluro_images.append(files[i])
    #print(Fluro_images)  ## print_out fluroscence image

    #Ch0 = res[0::2]
    for i in range(1, len(files),3):
        Grayscale_Fluro_images.append(files[i])
    #print(Grayscale_Fluro_images) ##print_out grayscale_image

    for i in range(2, len(files),3):
        Brightfield_images.append(files[i])
    #print(Brightfield_images)   

    files = Grayscale_Fluro_images
    z_indicator = '_z(\d\d)'
    regex_z = re.compile(z_indicator)

    def sort_key(file):
        return regex_z.search(file).group(1)

    files.sort(key=sort_key)
    brightfld_array = np.expand_dims(np.stack([color.rgb2gray(tifffile.imread(file)) for file in files]), axis=0) # stack all the sorted tiffiles and expand dim to create a "channel dim"
    # z-score normalize array.
    brightfld_array.shape

    files = Fluro_images
    z_indicator = '_z(\d\d)'
    regex_z = re.compile(z_indicator)

    def sort_key(file):
        return regex_z.search(file).group(1)

    files.sort(key=sort_key)
    fluroscense_array = np.expand_dims(np.stack([color.rgb2gray(tifffile.imread(file)) for file in files]), axis=0) # stack all the sorted tiffiles and expand dim to create a "channel dim"
    # z-score normalize array.
    fluroscense_array.shape #print(X3.shape)
    
    Mix_ch_24hr_X3= np.concatenate((fluroscense_array, brightfld_array ), axis=0)
    
    with writers.ome_tiff_writer.OmeTiffWriter(SaveIMG, overwrite_file=True) as writer:
        writer.save(Mix_ch_24hr_X3, dimension_order="CZYX", channel_names=['fluroscense','brightfield']) 
    

    print("\n\n2 Channel(BF,Flu)-Array[CZYX]")
    return  Mix_ch_24hr_X3.shape

In [5]:
# create the training set -- by converting desire image directory into ome.tif 
# call the function
# 1. prove the directory path and saved_image name "XYZ.ome.tif"

Get_ome_tif_file(files = glob.glob('Moffitt_Dataset/3hr_0.75Nuc_0um_V_1e5_630X4/*.tif') , SaveIMG = "tr_1/One-Ome_tif/3hr_0.75Nuc_0um_V_1e5_630X4.ome.tif")



2 Channel(BF,Flu)-Array[CZYX]


(2, 70, 1024, 1024)